## 02. Prétraitement des données

Ce notebook a pour objectif de mettre en œuvre les étapes de prétraitement du jeu de données eudirectlapse avant la modélisation. Il inclut la séparation des données en jeux
d’entraînement et de test, ainsi que la préparation des variables explicatives quantitatives et quantitatives. 

Dans ce projet, ces choix de prétraitement permettront d'entrainer plusieurs modèles de machine learning et d'évaluer l'impact du SMOTE sur les prédictions. 

### 1- Packages et chargement des données 

Dans cette section, nous importons les bibliothèques nécessaires et chargeons le jeu de données depuis le dossier `data/raw`.

In [57]:
from pathlib import Path
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [58]:
DATA_RAW = Path("..") / "data" / "raw"
DATA_FILE = "eudirectlapse.csv"

df = pd.read_csv(DATA_RAW / DATA_FILE)
TARGET_COL = "lapse"

### 2- Séparation de la variable cible et des variables explicatives

On distingue :
- la variable cible `lapse`, qui indique si le contrat est résilié (1) ou non (0) ;
- les variables explicatives `X`, qui décrivent les caractéristiques du souscripteur, du contrat, du véhicule et des informations tarifaires.

Cette séparation va nous permettre d'entraîner des modèles de classification et évaluer leur performance de chaque modèle.

In [62]:
X = df.drop(columns=[TARGET_COL]).copy()
y = df[TARGET_COL].copy()

print(X.shape, y.shape)
print(y.value_counts(normalize=True))

(23060, 21) (23060,)
lapse
0    0.871899
1    0.128101
Name: proportion, dtype: float64


### 3- Séparation jeu d'apprentissage et de test 

Les données sont séparées en un **jeu d’entraînement (75 %)** et un **jeu de test (25 %)**.

La séparation est réalisée de manière **stratifiée** selon la variable à expliquée (`stratify=y`) afin de conserver une proportion comparable de résiliations dans les deux jeux. Autrement dit, la distribution de la variable `lapse` dans les deux bases est la même, environ 12,8 %.

 `random_state = 42` est fixée afin de garantir la reproductibilité des résultats. 

In [63]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    stratify=y,
    random_state=42
)

print("Train target rate:", y_train.mean())
print("Test target rate:", y_test.mean())

Train target rate: 0.1281295172015033
Test target rate: 0.1280138768430182


### 4-Identification des variables numériques et catégorielles 

Les variables explicatives du jeu de données sont : soit numériques (par exemple âges, primes, ancienneté), soit catégorielles (par exemple région, genre, usage, type de garage).  

Nous identifions ces deux types de variables afin d’adapter le prétraitement et de préparer l’application de SMOTE.


In [64]:
num_cols = X_train.select_dtypes(include=["number", "bool"]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

print("variables numériques :", len(num_cols))
print("variables catégorielles :", len(cat_cols))

variables numériques : 12
variables catégorielles : 9


SMOTE est une méthode qui repose sur l'identification des plus proches voisins et permet la création de nouvelles observations synthétiques par interpolation linéaire. Cette méthode s'appuie sur le calcul de distances entre les observations, ce qui la rend principalement adaptée aux variables numériques. 

Dans ce projet, deux stratégies peuvent être envisagées : 
1. Appliquer SMOTE uniquement sur les variables numériques, 
2. Utiliser une variante de la méthode SMOTE qui permet de prendre en compte les variables catégorielles comme l'extention SMOTE-NC présentée par Chawla et al. (2002).

Dans la suite, nous retenons la première option, c'est à dire d'utiliser SMOTE uniquement sur les variables explicatives numériques afin d’appliquer SMOTE dans un cadre respectant ses hypothèses théoriques et d’éviter la création d’observations synthétiques peu interprétables pour les variables catégorielles.

In [65]:
#Permet d'isoler les variables numériques 
X_train_num = X_train[num_cols].copy()
X_test_num  = X_test[num_cols].copy()

print(X_train_num.shape, X_test_num.shape)


(17295, 12) (5765, 12)


Une fois les variables numériques isolées, le jeu de données comporte neuf variables explicatives, constituant l’espace des features utilisé pour l’application de SMOTE.

Les variables numériques sont standardisées (avec une moyenne à 0 et un écart type à 1)  afin d’éviter que certaines variables dominent le calcul des distances utilisées par SMOTE. Cette étape est particulièrement importante lorsque les variables n’ont pas les mêmes ordres de grandeur (par exemple une prime vs un âge).

In [66]:
scaler = StandardScaler()

X_train_num_scaled = scaler.fit_transform(X_train_num)
X_test_num_scaled  = scaler.transform(X_test_num)

### 5 - Sauvegarde des données 

In [67]:
# Dossier de sortie
OUT_DIR = Path("..") / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Sauvegarde
np.savez_compressed(
    OUT_DIR / "eudirectlapse_num_scaled_split.npz",
    X_train=X_train_num_scaled,
    X_test=X_test_num_scaled,
    y_train=y_train.to_numpy(),
    y_test=y_test.to_numpy(),
    num_cols=np.array(num_cols, dtype=object)
)

# Sauvegarde du scaler (pour reproduire la transformation)
joblib.dump(scaler, OUT_DIR / "scaler_num.joblib")

print(" Fichiers enregistrés dans:", OUT_DIR)


 Fichiers enregistrés dans: ..\data\processed


### Conclusion

Ce notebook a permis de mettre en place un prétraitement reproductible de la base de données. 

Dans le notebook suivant, nous entraînerons deux modèles de référence (avec et sans rééquilibrage par la méthode SMOTE), puis nous évaluerons l’impact de SMOTE sur la détection des résiliations.
